## HuggingFace Transformers 
#### Export Transformers models
- TorchScript
  

### 01、BERT Example test

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 36.0 MB/s 
     |████████████████████████████████| 101 kB 7.7 MB/s 


In [ ]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("bert-base-uncased")


# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

dummy_input = [tokens_tensor, segments_tensors]


# Initializing the model with the torchscript flag
# Flag set to True even though it is not necessary as this model does not have an LM Head.
# config = BertConfig(
#     vocab_size_or_config_json_file=32000,
#     hidden_size=768,
#     num_hidden_layers=12,
#     num_attention_heads=12,
#     intermediate_size=3072,
#     torchscript=True,
# )

# # Instantiating the model
# model = BertModel(config)

# The model needs to be in evaluation mode
model.eval()

# If you are instantiating the model with *from_pretrained* you can also easily set the TorchScript flag
model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

# Creating the trace
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])
torch.jit.save(traced_model, "traced_bert.pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
loaded_model = torch.jit.load("traced_bert.pt")
loaded_model.eval()

# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

dummy_input = [tokens_tensor, segments_tensors]

all_encoder_layers, pooled_output = loaded_model(*dummy_input)

In [ ]:
pooled_output = loaded_model(*dummy_input)

In [ ]:
print(pooled_output)

(tensor([[[-2.5689e-01, -7.3599e-03, -8.9147e-02,  ..., -1.3546e-01,
           2.3597e-01,  2.4208e-01],
         [-5.8262e-01,  3.1923e-01, -2.8020e-01,  ...,  1.0413e-01,
           1.7953e-01, -4.7086e-01],
         [-3.0671e-01, -2.3213e-01, -1.5938e-01,  ...,  7.0993e-02,
           1.4761e-01,  2.7529e-01],
         ...,
         [ 2.0549e-01, -1.6317e-02, -7.1004e-05,  ..., -1.3032e-01,
           6.1008e-01,  4.2999e-01],
         [-4.9530e-01, -4.6195e-01, -2.9027e-01,  ...,  6.3559e-01,
           6.2100e-01,  1.0318e-01],
         [ 8.2051e-01,  1.8250e-01, -1.1302e-01,  ...,  1.5103e-01,
          -7.6513e-01, -1.9481e-02]]], grad_fn=<DifferentiableGraphBackward>), tensor([[-4.9859e-01, -1.6913e-01,  8.3044e-01,  7.2490e-02, -4.8807e-01,
         -9.1259e-02,  5.1964e-01,  1.2615e-01,  7.3988e-01, -9.9609e-01,
          3.7945e-01, -5.8106e-01,  9.5275e-01, -6.8154e-01,  7.0220e-01,
         -2.4374e-01,  9.2702e-02, -3.1205e-01,  2.3801e-01, -3.3289e-01,
          2.2093e

In [ ]:
traced_model(tokens_tensor, segments_tensors)

### 02、Other test example

In [ ]:
import os
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono", torchscript=True)

max_length = 128
PAD = "<pad>"
tokenizer.pad_token = PAD

dummy_input =  "def hello_world():"

# Creating a dummy input
inputs = tokenizer.encode_plus(dummy_input,max_length = int(max_length), pad_to_max_length = True, truncation=True, return_tensors = 'pt')
input_ids = inputs["input_ids"]
traced_model = torch.jit.trace(model, [input_ids])

# Creating the trace
torch.jit.save(traced_model, "traced_codegen-350M-mono.pt")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/codegen/modeling_codegen.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask_value = torch.tensor(mask_value, dtype=attn_weights.dtype).to(attn_weig

In [ ]:
loaded_model = torch.jit.load("traced_codegen-350M-mono.pt")
loaded_model.eval()

all_encoder_layers, pooled_output = loaded_model(*dummy_input)

RuntimeError: ignored

### 03、T5-small  

In [ ]:
!pip install sentencepiece==0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small', torchscript = True)
input_ids = tokenizer('The <extra_id_0> walks in <extra_id_1> park', return_tensors='pt').input_ids
attention_mask = input_ids.ne(model.config.pad_token_id).long()
decoder_input_ids = tokenizer('<pad> <extra_id_0> cute dog <extra_id_1> the <extra_id_2>', return_tensors='pt').input_ids

traced_model = torch.jit.trace(model, (input_ids, attention_mask, decoder_input_ids))
torch.jit.save(traced_model, "traced_t5.pt")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:679: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


In [ ]:
input_ids = tokenizer('The <extra_id_0> walks in <extra_id_1> park', return_tensors='pt').input_ids
attention_mask = input_ids.ne(model.config.pad_token_id).long()
decoder_input_ids = tokenizer('<pad> <extra_id_0> cute dog <extra_id_1> the <extra_id_2>', return_tensors='pt').input_ids

dummy_input = [input_ids, attention_mask]

In [ ]:
loaded_model = torch.jit.load("traced_t5.pt")
loaded_model.eval()

all_encoder_layers, pooled_output = loaded_model(*dummy_input)

RuntimeError: ignored

## TorchServe + MNIST 测试案例

In [ ]:
!curl http://127.0.0.1:7000/predictions/mnist -T ./drive/MyDrive/NLP/TorchScript/mnist/mnist/test_data/4.png

4

In [13]:
!curl -X POST http://127.0.0.1:7000/predictions/codegen -H 'Content-Type: application/json' -d '{"data":"import matplot"}'

import matplotlib.pyplot as plt
import numpy as np
import pandas

In [35]:
inferences = []
outputs = 'import pandas as pd'

inferences = inferences.append(outputs)
print(inferences)

None


In [3]:
!curl -X POST http://127.0.0.1:7000/predictions/codegen-350M -H 'Content-Type: application/json' -d '{"data":"print hello world function"}'

{
  "code": 404,
  "type": "ModelNotFoundException",
  "message": "Model not found: codegen-350M"
}


In [14]:
!nvidia-smi

Fri Sep  2 09:34:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    36W / 250W |   2233MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
dic1 = {"main": "this is a main", "categories": "this is a categories"}
count = 2
input_data = [dic1 for i in range(count)]

In [2]:
print(input_data)

[{'main': 'this is a main', 'categories': 'this is a categories'}, {'main': 'this is a main', 'categories': 'this is a categories'}]


In [9]:
!curl -X POST http://127.0.0.1:7000/predictions/Textgeneration  -T  sample_text.txt

curl: (56) Recv failure: Connection reset by peer


In [ ]:
!pwd

/content


## Generative models T5

In [ ]:
!pip install transformers
!pip install SentencePiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.7 MB/s 


In [ ]:
text = """At a very high level, one of the most critical steps in any ML pipeline is called AI serving, a task usually performed by an AI inference engine. The AI inference engine is responsible for the model deployment and performance monitoring steps in the figure above, and represents a whole new world that will eventually determine whether applications can use AI technologies to improve operational efficiencies and solve real business problems. """

from transformers import T5ForConditionalGeneration, T5Tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


tensor([[    0,    80,    13,     8,   167,  2404,  2245,    16,   136,     3,
          6858, 12045,    19,   718,  7833,  3122,     3,     5,     8,  7833,
            16, 11788,  1948,    19,  1966,    21,     8,   825, 12001,    11,
           821,  4891,     3,     5,    34,    56,  2082,   823,  1564,    54,
           169,  7833,  2896,    12,  1172,  7763,     3, 27752,     3,     5,
             1]])
<pad> one of the most critical steps in any ML pipeline is called AI serving. the AI inference engine is responsible for the model deployment and performance monitoring. it will determine whether applications can use AI technologies to improve operational efficiencies.</s>


In [ ]:
# 无法进行有效的运行，生成输出

class MyModel(T5ForConditionalGeneration):
    def __init__(self, kwargs):
        super(MyModel, self).__init__(kwargs)

    def forward(self, params):
        self.generate(params)

newmodel = MyModel.from_pretrained("t5-base", torchscript=True)
output = newmodel(inputs)

In [ ]:
# 可以进行有效的输出，并使用束搜索进行文本生成

t_input = "translate English to French: The universe is a dark forest."

token = tokenizer(t_input, return_tensors="pt")

input_ids = token["input_ids"]
attention_mask = token["attention_mask"]
# 'set num_beams = 1' for greedy search
tokens = model.generate(input_ids=input_ids, attention_mask=attention_mask, num_beams=2)
output = tokenizer.decode(tokens.squeeze(), skip_special_tokens=True)

print(output)

L'univers est une forêt sombre.


In [ ]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5Model.from_pretrained("t5-base")

input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# forward pass
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
last_hidden_states = outputs.last_hidden_state

TypeError: ignored

## Using TorchScript in Python

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 120 kB 48.7 MB/s 
     |████████████████████████████████| 6.6 MB 27.9 MB/s 


In [ ]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

# Initializing the model with the torchscript flag
# Flag set to True even though it is not necessary as this model does not have an LM Head.
config = BertConfig(
    vocab_size_or_config_json_file=32000,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    torchscript=True,
)

# Instantiating the model
model = BertModel(config)

# The model needs to be in evaluation mode
model.eval()

# If you are instantiating the model with *from_pretrained* you can also easily set the TorchScript flag
model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

# Creating the trace
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])
torch.jit.save(traced_model, "traced_bert.pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

loaded_model = torch.jit.load("traced_bert.pt")
loaded_model.eval()

all_encoder_layers, pooled_output = loaded_model(*dummy_input)

In [ ]:
all_encoder_layers

tensor([[[-2.5689e-01, -7.3599e-03, -8.9147e-02,  ..., -1.3546e-01,
           2.3597e-01,  2.4208e-01],
         [-5.8262e-01,  3.1923e-01, -2.8020e-01,  ...,  1.0413e-01,
           1.7953e-01, -4.7086e-01],
         [-3.0671e-01, -2.3213e-01, -1.5938e-01,  ...,  7.0993e-02,
           1.4761e-01,  2.7529e-01],
         ...,
         [ 2.0549e-01, -1.6317e-02, -7.1004e-05,  ..., -1.3032e-01,
           6.1008e-01,  4.2999e-01],
         [-4.9530e-01, -4.6195e-01, -2.9027e-01,  ...,  6.3559e-01,
           6.2100e-01,  1.0318e-01],
         [ 8.2051e-01,  1.8250e-01, -1.1302e-01,  ...,  1.5103e-01,
          -7.6513e-01, -1.9481e-02]]], grad_fn=<NativeLayerNormBackward0>)

In [ ]:
pooled_output

tensor([[-4.9859e-01, -1.6913e-01,  8.3044e-01,  7.2490e-02, -4.8807e-01,
         -9.1259e-02,  5.1964e-01,  1.2615e-01,  7.3988e-01, -9.9609e-01,
          3.7945e-01, -5.8106e-01,  9.5275e-01, -6.8154e-01,  7.0220e-01,
         -2.4374e-01,  9.2702e-02, -3.1205e-01,  2.3801e-01, -3.3289e-01,
          2.2093e-01, -3.1050e-01,  7.1334e-01,  6.8511e-02,  1.5540e-01,
         -7.6150e-01, -2.8993e-01,  7.5329e-01,  8.3334e-01,  5.7146e-01,
         -3.3142e-01,  1.4806e-01, -9.5029e-01, -9.6337e-02,  7.7407e-01,
         -9.3747e-01, -3.0847e-02, -4.7886e-01,  4.4816e-02,  1.1357e-02,
         -6.1463e-01,  2.0118e-01,  9.0852e-01, -6.6917e-01, -3.0572e-01,
         -2.4733e-01, -6.9975e-01,  9.1703e-02, -6.5932e-01, -8.4444e-01,
         -7.1574e-01, -8.7206e-01,  5.5364e-02,  1.0259e-01,  2.0758e-01,
          4.1869e-01, -2.2090e-01,  6.6650e-02,  3.9337e-02, -3.4385e-01,
         -4.0217e-01,  3.1609e-02,  6.8003e-01, -7.3265e-01, -7.0735e-01,
         -8.4914e-01, -3.4449e-02, -7.

In [ ]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_ids = enc.encode(question, text)

all_tokens = enc.convert_ids_to_tokens(input_ids)
answer = ' '.join(all_tokens[torch.argmax(all_encoder_layers) : torch.argmax(pooled_output)+1])

print(answer)

In [ ]:
traced_model(tokens_tensor, segments_tensors)

(tensor([[[-2.5689e-01, -7.3599e-03, -8.9147e-02,  ..., -1.3546e-01,
            2.3597e-01,  2.4208e-01],
          [-5.8262e-01,  3.1923e-01, -2.8020e-01,  ...,  1.0413e-01,
            1.7953e-01, -4.7086e-01],
          [-3.0671e-01, -2.3213e-01, -1.5938e-01,  ...,  7.0993e-02,
            1.4761e-01,  2.7529e-01],
          ...,
          [ 2.0549e-01, -1.6317e-02, -7.1004e-05,  ..., -1.3032e-01,
            6.1008e-01,  4.2999e-01],
          [-4.9530e-01, -4.6195e-01, -2.9027e-01,  ...,  6.3559e-01,
            6.2100e-01,  1.0318e-01],
          [ 8.2051e-01,  1.8250e-01, -1.1302e-01,  ...,  1.5103e-01,
           -7.6513e-01, -1.9481e-02]]], grad_fn=<NativeLayerNormBackward0>),
 tensor([[-4.9859e-01, -1.6913e-01,  8.3044e-01,  7.2490e-02, -4.8807e-01,
          -9.1259e-02,  5.1964e-01,  1.2615e-01,  7.3988e-01, -9.9609e-01,
           3.7945e-01, -5.8106e-01,  9.5275e-01, -6.8154e-01,  7.0220e-01,
          -2.4374e-01,  9.2702e-02, -3.1205e-01,  2.3801e-01, -3.3289e-01,
    

In [ ]:
torch.jit.trace(model, [tokens_tensor, segments_tensors])

BertModel(
  original_name=BertModel
  (embeddings): BertEmbeddings(
    original_name=BertEmbeddings
    (word_embeddings): Embedding(original_name=Embedding)
    (position_embeddings): Embedding(original_name=Embedding)
    (token_type_embeddings): Embedding(original_name=Embedding)
    (LayerNorm): LayerNorm(original_name=LayerNorm)
    (dropout): Dropout(original_name=Dropout)
  )
  (encoder): BertEncoder(
    original_name=BertEncoder
    (layer): ModuleList(
      original_name=ModuleList
      (0): BertLayer(
        original_name=BertLayer
        (attention): BertAttention(
          original_name=BertAttention
          (self): BertSelfAttention(
            original_name=BertSelfAttention
            (query): Linear(original_name=Linear)
            (key): Linear(original_name=Linear)
            (value): Linear(original_name=Linear)
            (dropout): Dropout(original_name=Dropout)
          )
          (output): BertSelfOutput(
            original_name=BertSelfOutput


## 其他测试案例

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', torchscript=True)

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_ids = tokenizer.encode(question, text)
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

input_tensor = torch.tensor([input_ids])
token_type_ids_tensor = torch.tensor([token_type_ids])

# The way I traced the model could be wrong here
traced_model = torch.jit.trace(model, (input_tensor, token_type_ids_tensor))
traced_model.eval()
start_scores, end_scores = traced_model(input_tensor, token_type_ids_tensor)

all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

print(answer)

jim henson was a nice puppet [SEP]


## Torchserve  ONNX

In [ ]:
!pip install transformers[onnx]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.2 MB/s 
     |████████████████████████████████| 120 kB 83.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.2 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 212 kB 91.0 MB/s 
     |████████████████████████████████| 442 kB 86.6 MB/s 
     |████████████████████████████████| 4.9 MB 74.8 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 13.1 MB 25.9 MB/s 
     |████████████████████████████████| 99 kB 10.3 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=ed29ca27167ef201bff70ef8564c56a5fb4bc3a6317ce27ebcd303fba446d884
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/

In [ ]:
!python -m transformers.onnx --model=Salesforce/codegen-350M-mono onnx

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing CodeGenModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CodeGenModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using framework PyTorch: 1.12.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> False
/usr/local/lib/python3.7/dist-packages/transformers/models/codegen/modeling_codegen.py:509: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. Thi

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer
from onnxruntime import InferenceSession

tokenizer = AutoTokenizer.from_pretrained('Salesforce/codegen-350M-mono')
session = InferenceSession("onnx/model.onnx")

inputs = tokenizer("def hello_world(): ", return_tensors='np')
outputs = session.run(output_names=["last_hidden_state"], input_feed=dict(inputs))

In [ ]:
outputs

[array([[[-0.04647821, -0.47843856, -0.14148879, ...,  2.0196798 ,
           8.413227  ,  0.64264494],
         [-1.9656425 ,  0.434402  ,  1.315953  , ..., -1.6776477 ,
           1.4410083 , -0.25056827],
         [-2.9962916 ,  0.72312593,  2.411325  , ..., -2.591769  ,
          -2.3318453 , -2.0183125 ],
         [-4.301694  ,  2.9909708 , -2.0072517 , ..., -2.4425151 ,
           0.22334492, -2.0477571 ],
         [ 2.0143027 ,  0.76100916,  0.02836862, ...,  3.8227057 ,
           1.1650985 , -4.368134  ],
         [-0.18233556, -5.520864  ,  0.6300617 , ..., -3.2548928 ,
           3.958061  , -3.568727  ]]], dtype=float32)]

In [ ]:
print(inputs)

{'input_ids': array([[ 4299, 23748,    62,  6894, 33529,   220]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1]])}


In [ ]:
tokenizer.decode()

TypeError: ignored

In [ ]:
import numpy as np
from scipy.special import softmax
for i in range(6):
    m = np.argmax(softmax(outputs[0][0][i], axis=0))
    print(m)

919
97
10
133
97
189


In [ ]:
np.shape(outputs)

(1, 1, 6, 1024)

In [ ]:
print(tokenizer.convert_ids_to_tokens(919))
print(tokenizer.convert_ids_to_tokens(97))
print(tokenizer.convert_ids_to_tokens(189))

cess
¤
ā
